System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.11.5
Commit 760b2e5b739 (2025-04-14 06:53 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin24.0.0)
  CPU: 12 × Apple M2 Max
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, apple-m2)
Threads: 8 default, 0 interactive, 4 GC (on 8 virtual cores)
Environment:
  JULIA_NUM_THREADS = 8
  JULIA_EDITOR = code


Load packages:

In [3]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/github.com/ucla-biostat-257/2025spring/hw/hw5`


Status `~/Documents/github.com/ucla-biostat-257/2025spring/hw/hw5/Project.toml`
  [1e616198] COSMO v0.8.9
  [61c947e1] Clarabel v0.10.0
  [f65535da] Convex v0.16.4
  [a93c6f00] DataFrames v1.7.0
  [60bf3e95] GLPK v1.2.1
  [2e9cd046] Gurobi v1.7.4
  [87dc4568] HiGHS v1.17.0
  [b99e6be6] Hypatia v0.8.2
  [4076af6c] JuMP v1.25.0
  [67920dd8] KNITRO v0.14.8
  [1ec41992] MosekTools v0.15.9
  [2f354839] Pajarito v0.8.2
  [08abe8d2] PrettyTables v2.4.0
  [c946c3f1] SCS v2.1.0
  [3eaba693] StatsModels v0.7.4


In this exercise, we practice using disciplined convex programming (SDP in particular) to solve optimal design problems.

## Introduction to optimal design

Consider a linear model
\begin{eqnarray*}
	y_i = \mathbf{x}_i^T \boldsymbol{\beta} + \epsilon_i, \quad i = 1,\ldots, n,
\end{eqnarray*}
where $\epsilon_i$ are independent Gaussian noises with common variance $\sigma^2$. It is well known that the least squares estimate $\hat{\boldsymbol{\beta}}$ is unbiased and has covariance $\sigma^2 (\sum_{i=1}^n \mathbf{x}_i \mathbf{x}_i^T)^{-1}$. 

In **exact optimal design**, given total number of $n$ allowable experiments, we want to choose among a list of $m$ candidate design points $\{\mathbf{x}_1, \ldots, \mathbf{x}_m\}$ such that the covariance matrix is minimized in some sense. In mathematical terms, we want to find an integer vector $\mathbf{n} = (n_1, \ldots, n_m)$ such that $n_i \ge 0$, $\sum_{i=1}^m n_i = n$, and the matrix $\mathbf{V} = \left( \sum_{i=1}^m n_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1}$ is "small".

In **approximate optimal design**,  we want to find a probability vector $\mathbf{p} = (p_1, \ldots, p_m)$ such that $p_i \ge 0$, $\sum_{i=1}^m p_i = 1$, and the matrix $\mathbf{V} = \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1}$ is "small".

Commonly used optimal design criteria include:

- In **$D$-optimal design**, we minimize the determinant of $\mathbf{V}$
\begin{eqnarray*}
	&\text{minimize}& \det \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.
\end{eqnarray*}

- In **$E$-optimal design**, we minimize the spectral norm, i.e., the maximum eigenvalue of $\mathbf{V}$
\begin{eqnarray*}
	&\text{minimize}& \lambda_{\text{max}} \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.	
\end{eqnarray*}
Statistically we are minimizing the maximum variance of $\sum_{j=1}^p a_j \text{var}(\hat \beta_j)$ over all vectors $\mathbf{a}$ with unit norm.

- In **$A$-optimal design**, we minimize the trace of $\mathbf{V}$
\begin{eqnarray*}
	&\text{minimize}& \text{tr} \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.
\end{eqnarray*}
Statistically we are minimizing the total variance $\sum_{j=1}^p \text{var}(\hat \beta_j)$.

## Q1 (10 pts) 3x4 factorial design

A drug company ask you to help design a two factor clinical trial, in which treatment A has three levels (A1, A2, and A3) and treatment B has four levels (B1, B2, B3, and B4). Drug company also tells you that the treatment combination A3:B4 has undesirable side effects so we ignore this design point. 

Using dummy coding with A1 and B1 as the baseline levels, find the matrix $C$ with each row a unique design point.

## Q2 (30 pts) Find approximate optimal designs

Using semidefinite programming (SDP) software to find the approximate D-, E-, and A-optimal designs for this clinical trial.

Hint: This is what I got (using Hypatia solver), which may or may not be correct.

```
Approximate Optimal Design
┌───────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│ design_pt │   D_opt │   E_opt │   A_opt │ D_opt_n │ E_opt_n │ A_opt_n │
│    String │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├───────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│      A1B1 │   0.082 │   0.271 │   0.200 │   8.000 │  27.000 │  20.000 │
│      A2B1 │   0.082 │   0.150 │   0.101 │   8.000 │  15.000 │  10.000 │
│      A3B1 │   0.097 │   0.118 │   0.104 │  10.000 │  12.000 │  10.000 │
│      A1B2 │   0.082 │   0.058 │   0.086 │   8.000 │   6.000 │   9.000 │
│      A2B2 │   0.082 │   0.037 │   0.051 │   8.000 │   4.000 │   5.000 │
│      A3B2 │   0.097 │   0.059 │   0.068 │  10.000 │   6.000 │   7.000 │
│      A1B3 │   0.082 │   0.058 │   0.086 │   8.000 │   6.000 │   9.000 │
│      A2B3 │   0.082 │   0.037 │   0.051 │   8.000 │   3.000 │   5.000 │
│      A3B3 │   0.097 │   0.059 │   0.068 │  10.000 │   6.000 │   7.000 │
│      A1B4 │   0.109 │   0.079 │   0.106 │  11.000 │   8.000 │  10.000 │
│      A2B4 │   0.109 │   0.075 │   0.080 │  11.000 │   7.000 │   8.000 │
│       Obj │   8.987 │  13.000 │  38.925 │   8.988 │  13.028 │  38.946 │
└───────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┘
```

## Q3 (30 bonus pts) Find exact optimal designs

Using mixed-integer semidefinite programming (SDP) software to find the exact D-, E-, and A-optimal designs for this clinical trial **with $n=100$**.

Hint: This is what I got using Pajarito with Gurobi as the `oa_solver` and Mosek as the `conic_solver`.

```
Exact Optimal Design
┌───────────┬─────────┬─────────┬─────────┐
│ design_pt │   D_opt │   E_opt │   A_opt │
│    String │ Float64 │ Float64 │ Float64 │
├───────────┼─────────┼─────────┼─────────┤
│      A1B1 │   8.000 │  34.000 │  20.000 │
│      A2B1 │   8.000 │  10.000 │  10.000 │
│      A3B1 │  10.000 │  10.000 │  10.000 │
│      A1B2 │   8.000 │   9.000 │   9.000 │
│      A2B2 │   8.000 │   0.000 │   5.000 │
│      A3B2 │  10.000 │   6.000 │   7.000 │
│      A1B3 │   8.000 │   8.000 │   9.000 │
│      A2B3 │   8.000 │   4.000 │   5.000 │
│      A3B3 │  10.000 │   4.000 │   7.000 │
│      A1B4 │  11.000 │   9.000 │  10.000 │
│      A2B4 │  11.000 │   6.000 │   8.000 │
│       Obj │   8.988 │  13.011 │  38.946 │
└───────────┴─────────┴─────────┴─────────┘
```

## Q4 (30 points) Optimal design with nuisance parameters

Suppose the regression coefficients of linear model $\boldsymbol{\beta}$ is partitioned as $\boldsymbol{\beta} = (\boldsymbol{\beta}_0^T, \boldsymbol{\beta}_1^T)^T$, where $\boldsymbol{\beta}_0$ are nuisance parameters and $\boldsymbol{\beta}_1$ are parameters of primary interest. Given an approximate design $\mathbf{p} = (p_1, \ldots, p_m)$, let the information matrix be partitioned accordingly
$$
\mathbf{I}(\mathbf{p}) = \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T =  \begin{pmatrix}
\mathbf{I}_{00} & \mathbf{I}_{01} \\
\mathbf{I}_{10} & \mathbf{I}_{11}
\end{pmatrix}.
$$
Then the information matrix for $\boldsymbol{\beta}_1$ adjusted for nuisance parameter $\boldsymbol{\beta}_0$ is
$$
\mathbf{I}_{1 \mid 0}(\mathbf{p}) = \mathbf{I}_{11} - \mathbf{I}_{10} \mathbf{I}_{00}^{-1} \mathbf{I}_{01}.
$$

Revisiting the 3x4 factorial design problem in Q1, suppose the drug company only cares about the estimation of A treatment effects. Find the approximate D-, E-, and A-optimal designs.